In [ ]:
# 10/19/2021
# No need to run first section again. 
# Run import statements then proceed to Preprocess Data section where csv is loaded in.

# BINNING NEEDS WORK IN ORDER FOR MODELS TO RUN

!pip uninstall scikit-learn -y

!pip install -U scikit-learn

# !pip uninstall xgboost -y

# !pip install -U xgboost

In [2]:
# import dependencies
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
# from config import Password, Username, Host,DB
import pandas as pd

# Preprocess Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
movies_df = pd.read_csv("/content/drive/MyDrive/final project/dummy_data_frame.csv")

In [16]:
movies_df

,movie_id,rating,date,runtime,metascore,imdb_rating,boxoffice,budget,buzz,Adventure,Comedy,Family,Animation,Action,Sci-Fi,Mystery,Thriller,Biography,Drama,Sport,Fantasy,Crime,Romance,Music,Musical,Horror,History,War,Documentary,News,Western,Short,Unnamed: 32
0,1,PG,2006-12-22,108,48,6.4,250863268,110000000,6,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,G,2006-06-09,117,73,7.1,244082982,70000000,4,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,PG-13,2006-05-26,104,58,6.7,234362462,210000000,23,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,PG-13,2006-05-19,149,46,6.6,217536138,125000000,7,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,PG-13,2006-06-28,154,72,6.0,200120000,232000000,15,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,6800,TV-MA,2021-02-12,88,44,5.5,892802,750000,4,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1663,6803,R,2021-05-21,117,69,6.9,845464,1400000,24,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1664,6835,PG-13,2021-05-07,108,67,6.4,632542,15000000,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1665,6850,Not Rated,2021-06-18,101,65,6.9,546201,7000000,4,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [17]:
labels = ["100M", "400M", "800M"]
breaks = [0, 100000000, 400000000, 800000000]
target = pd.cut(movies_df["boxoffice"] , bins=breaks, labels=labels, include_lowest=True)

In [5]:
# separate out target: Box Office
# bin target data
!pip install jenkspy
import jenkspy
labels = ["0-10724389", "10724389-27367660", "27367660-46462469","46462469-67790117","67790117-92186262","92186262-120146040", 
            "120146040-153636354", "153636354-191719337", "191719337-228778661", "228778661-268488329","268488329-312433331", 
            "312433331-368065385", "368065385-459005868", "459005868-533720947", "533720947-760507625"]
labels_3 = ["65M", "202M", "760M"]
labels_4 = ["42M","121M","262M","760M"]
breaks = jenkspy.jenks_breaks(movies_df["boxoffice"], nb_class=4)
target = pd.cut(movies_df["boxoffice"] , bins=breaks, labels=labels_4, include_lowest=True)


     |████████████████████████████████| 55 kB 3.1 MB/s 
  Created wheel for jenkspy: filename=jenkspy-0.2.0-cp37-cp37m-linux_x86_64.whl size=70404 sha256=6bab63c6f8a511e07150d8e606b711429f03e0bb842b3cf918550e855c6dc51b
  Stored in directory: /root/.cache/pip/wheels/d8/d1/42/9bd05164ef706b84674905892f0542f8db6ac4a652473ca7e0
Successfully built jenkspy


In [18]:
breaks

[0, 100000000, 400000000, 800000000]

In [19]:

movies_df.drop(columns=["boxoffice"], inplace=True)

In [20]:
# clean data
# date into month
movies_df["month"] = pd.DatetimeIndex(movies_df['date']).month


In [21]:
# make dummies for rating
dummy_df = pd.get_dummies(movies_df, columns=["rating"])

In [22]:
dummy_df.drop(columns=["movie_id", "date"], inplace=True)

In [23]:
# split data into test train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dummy_df, target, random_state=1)

In [24]:
# scale data

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create Models

In [25]:
# random forest
# need to bin data for classifications
from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier(random_state=1, n_estimators=1000).fit(X_train_scaled, y_train)

In [26]:
print(f'Training Score: {random_forest_model.score(X_train_scaled, y_train)}')
print(f'Testing Score: {random_forest_model.score(X_test_scaled, y_test)}')

Training Score: 1.0
Testing Score: 0.8968824940047961


In [27]:
# neural network
import tensorflow as tf
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=100, activation="sigmoid", input_dim=40))

nn_model.add(tf.keras.layers.Dense(units=100, activation="sigmoid"))
nn_model.add(tf.keras.layers.Dense(units=100, activation="sigmoid"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 24,401
Trainable params: 24,401
Non-trainable params: 0
_________________________________________________________________


In [28]:
# neural net
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# neural net
# Fit the model to the training data
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [30]:

fit_model = nn_model.fit(X_train_scaled, le.transform(y_train), epochs=200)

Epoch 1/200
40/40 [==============================] - 1s 2ms/step - loss: 0.4526 - accuracy: 0.8496
Epoch 2/200
40/40 [==============================] - 0s 2ms/step - loss: 0.4275 - accuracy: 0.8496
Epoch 3/200
40/40 [==============================] - 0s 2ms/step - loss: 0.4152 - accuracy: 0.8496
Epoch 4/200
40/40 [==============================] - 0s 2ms/step - loss: 0.3915 - accuracy: 0.8496
Epoch 5/200
40/40 [==============================] - 0s 2ms/step - loss: 0.3389 - accuracy: 0.8544
Epoch 6/200
40/40 [==============================] - 0s 2ms/step - loss: 0.2910 - accuracy: 0.8808
Epoch 7/200
40/40 [==============================] - 0s 2ms/step - loss: 0.2554 - accuracy: 0.8928
Epoch 8/200
40/40 [==============================] - 0s 2ms/step - loss: 0.2429 - accuracy: 0.8944
Epoch 9/200
40/40 [==============================] - 0s 2ms/step - loss: 0.2405 - accuracy: 0.8952
Epoch 10/200
40/40 [==============================] - 0s 2ms/step - loss: 0.2384 - accuracy: 0.8960
Epoch 11/

In [19]:
import sklearn
sklearn.__version__

'0.22.2.post1'

In [31]:
# xg boost
from xgboost import XGBClassifier

# fit model no training data
xg_model = XGBClassifier()
xg_model.fit(X_train_scaled, y_train)


XGBClassifier(objective='multi:softprob')

In [32]:
# xg boost
# make predictions for test data
y_pred = xg_model.predict(X_test_scaled)
predictions = [value for value in y_pred]

from sklearn.metrics import accuracy_score
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 89.69%


# Output Models

In [36]:
import pickle
pickle.dump(xg_model, open("/content/drive/MyDrive/final project/new_xg_model.pkl", 'wb'))
pickle.dump(random_forest_model, open("/content/drive/MyDrive/final project/new_rf_model.pkl", 'wb'))

from keras.models import load_model

nn_model.save('/content/drive/MyDrive/final project/nn_model.h5')

In [37]:
from keras.models import load_model
model = load_model('/content/drive/MyDrive/final project/nn_model.h5')

In [39]:
model.predict([[155,75.0,8.4,21000000,4,1,0,0.0,0,1,0.0,0.0,0.0,0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,0.0,0.0,0.0,0,1,0,0.0,0.0,0.0,0.0],
[106,68.0,6.3,2100000000,4,1,1,0.0,1,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,0.0,0.0,0.0,1,0,0,0.0,0.0,0.0,0.0],
[106,98.0,8.0,21000000,4,0,0,0.0,0,0,0.0,0.0,0.0,0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0],
[111,85.0,6.4,21000000,4,0,0,0.0,0,0,0.0,0.0,0.0,1,1,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0],
[110,69.0,6.2,21000000,4,0,0,0.0,0,0,0.0,0.0,0.0,0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0],
[108,78.0,7.3,21000000,4,0,0,0.0,0,0,0.0,0.0,0.0,0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0],
[128,78.0,7.0,21000000,4,0,1,0.0,0,0,0.0,0.0,0.0,0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0],
[105,66.0,7.0,21000000,4,0,0,0.0,0,0,0.0,0.0,0.0,0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0]])

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)